In [6]:
!pip install fugashi unidic_lite
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 10.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import pipeline
import numpy as np
import torch

In [3]:
classifier = pipeline("zero-shot-classification",
                      model="Formzu/bert-base-japanese-jsnli",
                      device=torch.device("mps")

In [8]:
# premise = "この商品は肌の潤いを保ち、シワを減らします。"
# labels = ['化粧品', '食品', 'テレビ番組', '金融サービス']
# out = classifier(premise, labels, hypothesis_template="この例は{}です。")
# print(out)

In [7]:
from pathlib import Path
import json
import random
from sklearn.utils import shuffle

cmset = set()
noncmset = set()

def LoadDataset(path: Path, random_state: int=42):
    data = []
    with path.open('r') as f:
      dataset = json.load(f)
    for programItem in dataset:
      for text in programItem['cm']:
        if not text in cmset:
          cmset.add(text)
          data.append({
            'name': programItem['name'],
            'genre': programItem['genre'],
            'text': text,
            'label': 1
            })

      for text in programItem['noncm']:
        if not text in noncmset:
          noncmset.add(text)
          data.append({
            'name': programItem['name'],
            'genre': programItem['genre'],
            'text': text,
            'label': 0
          })
    data = shuffle(data, random_state=random_state)
    return shuffle(data, random_state=random_state)

dataset = LoadDataset(Path(r'speech_2025_30s.json'))
print(f'dataset size: {len(dataset)}')

dataset size: 24572


In [8]:
from tqdm import tqdm
from transformers import pipeline
import torch

# Ensure the pipeline is defined if it was lost in previous cell executions
#classifier = pipeline("zero-shot-classification", model="Formzu/bert-base-japanese-jsnli", device=0 if torch.cuda.is_available() else -1)

def predict(premise, categories):
  # Manually truncate the premise to a length slightly less than the max_length
  # A value like 510 or 511 is safe to avoid potential edge cases
  max_model_length = 512
  if len(premise) > max_model_length:
      premise = premise[:max_model_length - 2] # Keep some buffer for special tokens like [CLS], [SEP]

  out = classifier(premise, categories, hypothesis_template="{}", truncation=True, max_length=max_model_length)
  return out['labels'][0]

right = []
error = []
empty = []

with tqdm(dataset) as pbar:
  for dataItem in pbar:
    premise = dataItem['text']
    name = dataItem['name']
    genre = dataItem['genre']
    #categories = [f'これはテレビ{genre}{name}の一部です', f'これはある日用品、飲料、食品、サービスのテレビ広告です。']
    categories = ['これはドラマ、アニメ、またはドキュメンタリーの一部です。', 'これはある日用品、飲料、食品、サービスのテレビ広告の台本です']
    if premise == '':
      empty.append(dataItem)
      continue
    category = predict(premise=premise, categories=categories)
    if (category == categories[0] and dataItem['label'] == 0) or (category != categories[0] and dataItem['label'] == 1):
      right.append(dataItem)
    else:
      error.append(dataItem)
    acc = len(right) / (len(right) + len(error))
    pbar.set_postfix(acc=f"{acc:.2%}")

print(f'right: {len(right)}, error: {len(error)}, empty: {len(empty)}')

100%|███████████████████████████████████████████████████████████████████████████| 24572/24572 [39:40<00:00, 10.32it/s, acc=84.95%]

right: 20875, error: 3697, empty: 0


In [ ]:
for dataItem in error[:10]:
  premise = dataItem['text']
  name = dataItem['name']
  genre = dataItem['genre']
  categories = ['これはドラマ、アニメ、またはドキュメンタリーの一部です。', 'これはある日用品、飲料、食品、サービスのテレビ広告の台本です']
  out = predict(premise, categories)
  print(premise)
  print(out)
  print()